# Hyperparameter Optimization

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
import pandas as pd
import numpy as np

from sklearn.experimental import enable_halving_search_cv

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV , HalvingGridSearchCV, HalvingRandomSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import get_scorer_names, f1_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA


from scipy.stats import beta, loguniform

In [23]:
# data preparation
data = pd.read_csv('data/breast.data', header=None)
X = data.loc[:,2:].values
y = data.loc[:,1].map({'M':1, 'B':0}).values

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=1)

In [24]:
std_scaler = StandardScaler()
X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.fit_transform(X_test)

### Grid Search

The class implementing Grid search in SKLearn is GridSearchCV, from the sklearn.model_selection module.
The first step is to define the values which each hyperparameter can assume and define a grid, usually through a dictionary.

Important: the keys of the dictionary correspond to the names of the hyperparameters of the classifier, i.e. the parameters of the constructor. In the case of Pipeline objects the game is harder, since we have to use a special syntax for indicating the parameters of a specific element in the pipeline.
Here we evaluate the performance acting only on the classifier hyperparameter, LogisticRegression in this case. 

In [25]:
param_grid = {
    'penalty': ['l2', 'l1'],
    'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
}

Then, we define the classifier. Since the candidates for the penalty parameter are l1 and l2, we have to change the default solver for Logistic Regression, from 'lbfgs' to 'liblinear'.

In [26]:
cls = LogisticRegression(solver='liblinear')

In [27]:
gs = GridSearchCV(estimator=cls, 
                  param_grid=param_grid,
                  scoring='f1',
                  refit=True,
                  cv=10,
                  verbose=0)

In [28]:
gs = gs.fit(X_train_scaled, y_train)
print(f'Best score we got from the best estimator: {gs.best_score_}')
print(f'Configuration for the best estimator/classifier: {gs.best_params_}')

Best score we got from the best estimator: 0.9759145021645022
Configuration for the best estimator/classifier: {'C': 1.0, 'penalty': 'l2'}


GridSearchCV uses k-fold cross-validation for comparing the models associated with the different hyperparameter configurations. As for cross-validation, we can specify the performance metric for selecting the best classifier. Here, we used F1-score.

After cross-validation, we can get the score for the best fitting configuration by attribute best_score_ and the corresponding hyparams by the attribute best_params_.

Finally, by the attribute best_estimator_, we get the predictor object which got the best performance in CV. 
We don't need to re-train the model, because it's already done by GridSearchCV by default (parameter refit).

In [29]:
f1_score(y_test, gs.best_estimator_.predict(X_test_scaled))

0.9647058823529412

Here, we computed the F1-score on the test set using the best estimator returned by GridSearchCV.

In SKLEarn, the randomised grid search is implemented by the class RandomizedSearchCV in the module sklearn.model_selection.

In this case, we have to specify how we sample the values for each hyperparam. For each hyperparam, we define a probability distribution used for the sampling.

Suppose we have 3 hyperparameters p1, p2, p3, distributed according to P1, P2, P3, respectively. At each step, we extract a value from P1, a value from P2 and a value from P3 and make a triple of hyperparams. Extractions are independent.

The main difference with GridSearchCV is that we have to specify distributions. in the following example, we use Perceptron to show how to define a probability distribution on a parameter (learning rate, eta) and a uniform distribution over a list of values (epochs). For the learning rate (eta) we choose the Beta distribution.

In general, we can use any object which implements the method rvs(). All the distributions in scipy.stats fullfill this requirement.

In [30]:
beta(2,2).rvs(10)

array([0.78543593, 0.16139742, 0.37870956, 0.34458664, 0.34938546,
       0.37593123, 0.89196078, 0.73004436, 0.60463728, 0.4797088 ])

In [31]:
cls = Perceptron()
param_grid = {
    'eta0': beta(2, 2),
    'max_iter': [10, 30, 40, 100, 500, 1000]
}

In [32]:
rs = RandomizedSearchCV(estimator=cls,
                        param_distributions=param_grid,
                        scoring='f1',
                        refit=True,
                        n_iter=20,
                        cv=10,
                        random_state=1,
                        n_jobs=-1,
                        verbose=1)
rs = rs.fit(X_train_scaled, y_train)
print(f'Best score we got from the best estimator: {rs.best_score_}')
print(f'Configuration for the best estimator/classifier: {rs.best_params_}')

Fitting 10 folds for each of 20 candidates, totalling 200 fits


/Users/aleksandrak/Desktop/ML-course-Labs/.venv/lib/python3.12/site-packages/sklearn/linear_model/_stochastic_gradient.py:723: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/aleksandrak/Desktop/ML-course-Labs/.venv/lib/python3.12/site-packages/sklearn/linear_model/_stochastic_gradient.py:723: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/aleksandrak/Desktop/ML-course-Labs/.venv/lib/python3.12/site-packages/sklearn/linear_model/_stochastic_gradient.py:723: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/aleksandrak/Desktop/ML-course-Labs/.venv/lib/python3.12/site-packages/sklearn/linear_model/_stochastic_gradient.py:723: ConvergenceWarning: Maximum number of iteration reached before con

Best score we got from the best estimator: 0.9637770562770562
Configuration for the best estimator/classifier: {'eta0': 0.827280711292405, 'max_iter': 10}


We'll now use random grid search with Logistic Regression to verify if this search can identify a better configuration not taken into account by the grid search.

At each iteration (step), the size of training set increases (n_samples) and the number of candidate configurations decreases (n_candidates).

In SKLearn SH search is implemented by the classes HalvingSearchCV and HalvingRandomSearchCV. Both classes are still experimental, so we have to enable them.

In [33]:
cls = LogisticRegression(solver='liblinear')
param_grid = {
    'C': loguniform(0.0001, 1000),
    'penalty':['l1', 'l2']
}

In [34]:
hs = HalvingRandomSearchCV(
    cls, 
    param_distributions=param_grid,
    n_candidates='exhaust',
    resource='n_samples',
    factor=1.2,
    random_state=1,
    n_jobs=-1
)

hs = hs.fit(X_train_scaled, y_train)
print(f'Best score we got from the best estimator: {hs.best_score_}')
print(f'Configuration for the best estimator/classifier: {hs.best_params_}')

Best score we got from the best estimator: 0.9643835616438355
Configuration for the best estimator/classifier: {'C': 0.013071577689307423, 'penalty': 'l2'}


In [35]:
100 - 100/1.2

16.666666666666657

By the factor parameter we determine how many candidates are eliminated in each iteration: 100% - 100% / factor.

Via the resource parameter we specify which is the resource we increment at each iteration.

By n_candidates, we determine the number of candidate configurations in the first round. The value exhaust indicates that the number of candidates in the last round will be evaluated on the entire training set.

In [36]:
f1_score(y_test, hs.best_estimator_.predict(X_test_scaled))

0.9647058823529412

In the following we apply nested cross-validation selecting Logistic Regression as classifier, and in the inner loop we use randomized grid search strategy to find the best hyperparameter.

In [37]:
param_grid = {
    'C': loguniform(0.0001, 1000),
    'penalty': ['l1', 'l2']
}

hs_log = RandomizedSearchCV(estimator=LogisticRegression(solver='liblinear'),
                            param_distributions=param_grid,
                            scoring='f1',
                            n_iter=50,
                            cv=2,
                            verbose=1
)

scores = cross_val_score(hs_log, X_train_scaled, y_train,
                         scoring='f1', cv=5, verbose=1)
print(f'CV F1-score: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')

Fitting 2 folds for each of 50 candidates, totalling 100 fits
Fitting 2 folds for each of 50 candidates, totalling 100 fits
Fitting 2 folds for each of 50 candidates, totalling 100 fits
Fitting 2 folds for each of 50 candidates, totalling 100 fits
Fitting 2 folds for each of 50 candidates, totalling 100 fits
CV F1-score: 0.960 +/- 0.027


Now, it's time to test a KNN classifier, optimizing on K. In SKLearn the KNN classifier is implemented by the class KNeighborsClassifier in the sklearn.neighbors module.

In [38]:
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17]
}

hs_knn = RandomizedSearchCV(estimator=KNeighborsClassifier(),
                            param_distributions=param_grid,
                            scoring='f1',
                            cv=2,
                            verbose=1
)

scores = cross_val_score(hs_knn, X_train_scaled, y_train,
                         scoring='f1', cv=5, verbose=1)
print(f'CV F1-score: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')

Fitting 2 folds for each of 9 candidates, totalling 18 fits
Fitting 2 folds for each of 9 candidates, totalling 18 fits
Fitting 2 folds for each of 9 candidates, totalling 18 fits
Fitting 2 folds for each of 9 candidates, totalling 18 fits
Fitting 2 folds for each of 9 candidates, totalling 18 fits
CV F1-score: 0.951 +/- 0.040


### Hyperparameter optimization on a pipeline

All the above strategies can generalize to Pipeline objects with Predictor in the final step of the pipeline. The main difference w.r.t. the above examples is that we need a way to indicate the Hyperparameters of the different elements in the pipeline.

In this case, we have to remember that each element in a pipeline has an identifier - a string - associated to its Transformer/Predictor object. This way we just nee da syntax to indicate a hyperparameter name belonging to a specific identifier.

In SKLearn, we use the string _ to indicate this dependency.
For instance, given a Pipeline:

p = Pipeline(
    ('dim_reducer',PCA()),
    ('classifier',LogisticRegression())
)

we select the parameter n_components of the PCA object through this syntax:


dim_reducer__n_components

Using this syntax, we may run a (randomized) grid search on the entire pipeline, putting into the hyperparameter space all the yperparameters of the elements in the pipeline.

In [42]:
pipeline_log = Pipeline([
    ('scaling', StandardScaler()),
    ('feat_selection', PCA()),
    ('classifier', LogisticRegression())
])

params = [
    {
        'feat_selection__n_components': [0.3, 0.5, 0.7, 0.9, 1],
        'classifier__penalty': ['l1'],
        'classifier__C': loguniform(0.0001, 1000),
        'classifier__solver': ['libllinear']
    },
    {
        'feat_selection__n_components': [0.3, 0.5, 0.7, 0.9, 1],
        'classifier__penalty': ['l2'],
        'classifier__C': loguniform(0.0001, 1000)
    }
]

In [43]:
rs = RandomizedSearchCV(estimator=pipeline_log,
                        param_distributions=params,
                        scoring='f1',
                        refit=True,
                        n_iter=50,
                        cv=10,
                        random_state=1,
                        n_jobs=-1,
                        verbose=1)
rs = rs.fit(X_train, y_train)
print(f'Best score we got from the best estimator: {rs.best_score_}')
print(f'Configuration for the best estimator/classifier: {rs.best_params_}')

Fitting 10 folds for each of 50 candidates, totalling 500 fits
Best score we got from the best estimator: 0.9316436065985402
Configuration for the best estimator/classifier: {'classifier__C': 374.8819462573172, 'classifier__penalty': 'l2', 'feat_selection__n_components': 0.5}


In [44]:
f1_score(y_test, rs.best_estimator_.predict(X_test))

0.9382716049382716